## Data Partition

##### Import necessary modules

In [1]:
from pyspark.sql import SparkSession
from utils.Constants import Constants
from pyspark.sql.functions import month
from os.path import abspath
from pyspark.sql.functions import month, year, concat_ws

##### Configure the hadoop dir path

In [2]:
BASE_PATH = 'hdfs://localhost:9000/user/hadoop'
CLEANED_PATH = f'{BASE_PATH}/cleaned'

print('cleand path {}'.format(CLEANED_PATH))

cleand path hdfs://localhost:9000/user/hadoop/cleaned


##### Read the data that we have cleaned

In [3]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .getOrCreate()

# Path to the CSV file on HDFS
file_path = f'{CLEANED_PATH}/cleaned_data.csv'
schema = Constants.WEATHER_DATA_SCHEMA

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True , schema=Constants.WEATHER_DATA_SCHEMA)

df.printSchema()


24/12/27 11:42:21 WARN Utils: Your hostname, acer resolves to a loopback address: 127.0.1.1; using 192.168.0.119 instead (on interface wlp0s20f3)
24/12/27 11:42:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 11:42:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- water_content: float (nullable = true)
 |-- solar_radiation: float (nullable = true)
 |-- rain: float (nullable = true)
 |-- temperature: float (nullable = true)
 |-- rh: float (nullable = true)
 |-- wind_speed: float (nullable = true)
 |-- gust_speed: float (nullable = true)
 |-- wind_direction: float (nullable = true)
 |-- dew_point: float (nullable = true)



##### Let partition by date , becasue the col. date is not having high cardinality.

In [18]:
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# Read the CSV file with the defined schema
file_path = f'{CLEANED_PATH}/cleaned_data.csv'

# Read the CSV file into a DataFrame
data = spark.read.csv(file_path, header=True , schema=schema)
spark.sql("CREATE DATABASE IF NOT EXISTS mydb")

data = data \
    .withColumn("year_month", concat_ws("-", year(data["date"]), month(data["date"]))) \
    .orderBy("date", "time")

data.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("year_month") \
    .saveAsTable("mydb.weather_data_partitioned_by_month")

print(f"Data has been partitioned by month and saved to Hive table 'mydb.weather_data_partitioned_by_month'.")


24/12/18 22:49:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Data has been partitioned by month and saved to Hive table 'mydb.weather_data_partitioned_by_month'.


##### Describe the table schema

In [16]:
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("DESCRIBE mydb.weather_data_partitioned_by_month").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                date|   string|   NULL|
|                time|   string|   NULL|
|       water_content|    float|   NULL|
|     solar_radiation|    float|   NULL|
|                rain|    float|   NULL|
|         temperature|    float|   NULL|
|                  rh|    float|   NULL|
|          wind_speed|    float|   NULL|
|          gust_speed|    float|   NULL|
|      wind_direction|    float|   NULL|
|           dew_point|    float|   NULL|
|          year_month|   string|   NULL|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|          year_month|   string|   NULL|
+--------------------+---------+-------+



24/12/18 22:46:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


##### Test Load data

In [13]:
result = spark.sql("""
SELECT * 
FROM mydb.weather_data_partitioned_by_month
""")
result.show(5)


24/12/18 22:47:00 ERROR Executor: Exception in task 0.0 in stage 20.0 (TID 135)
org.apache.spark.SparkFileNotFoundException: File file:/home/panha/Desktop/big_data/rain_forecasting/spark-warehouse/mydb.db/weather_data_partitioned_by_month/year_month=2021-8/part-00003-42d9e529-05c0-4eeb-845c-baa7947ac5ca.c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.ha

Py4JJavaError: An error occurred while calling o187.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 135) (192.168.0.121 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/panha/Desktop/big_data/rain_forecasting/spark-warehouse/mydb.db/weather_data_partitioned_by_month/year_month=2021-8/part-00003-42d9e529-05c0-4eeb-845c-baa7947ac5ca.c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.s

In [14]:
spark.sql("SHOW TABLES IN mydb").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|     mydb|weather_data_part...|      false|
+---------+--------------------+-----------+

